# Week5 笔记
本周课程的内容分成3部分，包括涉及Bias/Variance、几种常见的正则化操作和一些实现DNN时必要的方法。
- Setting up your Machine Learning Application
- Regularizing your neural network
- Setting up your optimization problem

## Setting up your Machine Learning Application
这部分内容包括
- Train/Dev/Test sets
- Bias/Variance
- Basic Recipe for Machine Learning

**视频的一开始，NG不断地强调了一点：应用ML是一个不断迭代的过程！不断轮回！**

接下里整理知识点
##### 数据集的划分
以往ML中使用的数据集都比较小，划分时候经常按60% 20% 20%或者70% 15% 15%来划分训练集、验证集和测试集。但DL中由于数据集往往都比较大，所以不这么划分，留给验证集和测试集的比例都很小，大约是98% 1% 1%来划分。比如1,000,000的数据集，留10,000做验证，留10,000做测试即可。  
此外，一些人在划分数据集时会只画出训练集和验证集，即把验证集当作测试集用，这会导致过拟合的情况发生（**思考：为什么会过拟合？**）
##### 数据集的分布
从概率上解释，我们所做的最小化代价函数本质就是在求取给定$x$下$y$的条件概率分布（一般不求全概率分布）。在选取模型时候，我们通过验证集进行模型选择，通过测试集测试模型性能，**所以要求验证集和测试集是同分布的**。否则根据验证集选出来的模型在测试集上的表现可能会很差。
##### Bias和Variance
首先，明确Bias和Variance的意义。简单来理解，Bias就是欠拟合，而Variance则是过拟合。在ML中经常会强调Bias和Variance的平衡（Trade-off），因为在ML中没有工具能够做到只降低Bias或Variance中的一个而不影响另一个。而在DL中，能够实现只降低Bias而不增加Variance或是降低Variance而不增加Bias。因此，在DL中就不再提两者的平衡了。
##### 根据Bias和Variance选择后续方法
首先根据训练集和验证集确定Bias和Variance，判断是High Bias还是High Variance：
1. High Bias情况：
    - 增大网络规模
    - 增加训练时间/迭代次数
    - 更新网络结构
2. High Variance情况：
    - 收集更多数据
    - 采用正则化策略
    - 更新网络结构

## Regularizing your neural network
这部分内容包括
- Regularization
- Why regularization reduces overfitting
- Dropout regularization
- Understanding dropout
- Other regularization methods

**由于网络的加深，使得其模型容量增加，更容易产生过拟合的现象。因此，需要采用一些方法避免过拟合。这里重点学会正则化因子和drop out两种方法。当然，如果数据集非常非常大，过拟合的现象会大大降低。**(所以，请问数据集多大才算大？NG在视频里经常提到的数据集大小是1,000,000、5,000,000)**正则化除了降低过拟合的作用外，还能帮助降低误差。**

接下里开始内容总结

##### 正则化因子
通常网络学习时候的代价函数是
$$J(w,b)=\frac{1}{m}\sum_{i=1}^m L(\hat{y}^{(i)},\hat{y}) \tag{1}$$
引入$L_{2}$正则化，代价函数变为
$$J(w,b)=\frac{1}{m}\sum_{i=1}^m L(\hat{y}^{(i)},\hat{y})+\frac{\lambda}{2m}\lVert{w}\rVert_{2}^{2} \tag{2}$$
或是写成
$$J(w,b)=\frac{1}{m}\sum_{i=1}^m L(\hat{y}^{(i)},\hat{y})+\frac{\lambda}{2m}\lVert{w}\rVert_{F}^{2} \tag{3}$$
引入$L_{1}$正则化，代价函数变为
$$J(w,b)=\frac{1}{m}\sum_{i=1}^m L(\hat{y}^{(i)},\hat{y})+\frac{\lambda}{2m}\lVert{w}\rVert_{1} \tag{4}$$
以上两个公式有几点提醒：
1. $L_{2}$正则化项有平方，$L_{1}$正则化项没有平方
2. 正则化项只对$w$作用，而不对$b$作用，因为DNN中w的维数更高，且实践证明对$b$加不加正则化对最后结果基本没有影响
3. $L_{1}$正则化能帮助模型获得稀疏性，但实际上作用不大，相反的$L_{2}$正则化更加常用
4. 在引入正则化项的训练过程中观察代价函数随迭代次数的变化时，应当完整计算上面式子，即代价函数中应当包含正则化项

##### $L_{2}$正则化的解释
1. 相当于引入了一个权重系数
2. 相当于引入了罚因子

##### Drop out
Drop out实际上就是设定一个概率阈值$P_{keep}$，同时每个神经元会有一个激活概率$P_{act}$(即随机产生的一个0到1的数，用来和$P_{keep}$比较，则
$$f(x)=
\begin{cases}
神经元激活& P_{keep}\ge P_{act}\\
神经元抑制& P_{keep}<P_{act}
\end{cases} \tag{5}$$
所谓抑制，即指既不接受输入，也不产生输出。
具体流程可用伪代码表示如下：

|Algorithm 1|
|:-|
|set $P_{keep}$|
|for i=1 to max_num_iteration:|
|&emsp;&emsp;drop out|
|&emsp;&emsp;train samples|
|end|

|Algorithm 2 Drop out|
|:-|
|$d^{[l]}$=np.random.randn($n_{[l]}$,m)<1|
|$a^{[l]}$=$g(z^{[l]})$.*$d^{[l]}$|
|$z^{[l+1]}=W^{[l+1]}a^{[l]}$|
|...|
|$da^{[l]}=(W^{[l+1]})^{T}dz^{l+1}.*d^{[l]}$|
|$dz^{[l]}=da^{[l]}.*g'(z^{[l]})$|
|...|

在实现drop out时的几个**细节**：
1. 前向传播中，各层的激活值在drop out后要除以$P_{keep}$,因为由于drop out的作用一部分神经元被抑制，相当于用于向下一层传播的激活值$W^{[l+1]}a^{[l]}$减少$1-P_{keep}$的比例，所以为了避免激活衰减，要除以$P_{keep}$
2. 反向传播中，各层的$da^{[l]}$也要除以$P_{keep}$，因为原来应该分摊到各个神经元上的$da^{[l]}=(W^{[l+1]})^{T}dz^{[l+1]}$由于$l$层上某些神经元被抑制，使得用于修正$W^{[l+1]}$的$da^{[l]}$减小，所以也要除以$P_{keep}$
3. drop out只能在train时候用，不能在dev或是test时候用(用的话可以理解为对输出$\hat{y}$加入了噪声，非要用的话则需要重复很多次对$\hat{y}$取平均，结果会一样但是很低效)
4. 表达对神经元的抑制，只需要再定义一个元素值是0或1的矩阵$d^{[l]}$，通过和$A$和$dA$作用就可以来表示对$l$层神经元的激活或是抑制关系
5. 在绘制代价函数$J$随迭代次数的变化曲线时，建议关掉drop out方法求出$J$，然后再打开$J$进行下一次迭代训练
6. 最好不要对输入使用drop out，这样做会损失特征。根据经验，也不一定对每层都是用drop out，比如实践中输出的前一层使用drop out等
7. **Ng建议**：除非过拟合，否则最好不要使用drop out

## Setting up your optimization problem